In [ ]:
!pip install selenium

In [ ]:
!pip install webdriver_manager

In [ ]:
!pip install bs4

In [ ]:
!pip install pytesseract

In [4]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys

In [5]:
from bs4 import BeautifulSoup
import base64

In [6]:
def get_captcha_image():
    img_base64 = driver.execute_script("""
        var ele = arguments[0];
        var cnv = document.createElement('canvas');
        cnv.width = 215; cnv.height = 80;
        cnv.getContext('2d').drawImage(ele, 0, 0);
        return cnv.toDataURL('image/jpeg').substring(22);    
        """, driver.find_element_by_xpath('//*[@id="myForm:captchaImage"]'))   #"/html/body/form/div[2]/div[3]/span[3]/div[1]/img"))
    with open(r"image.jpg", 'wb') as f:
        f.write(base64.b64decode(img_base64)) 

In [7]:
from PIL import Image
from pytesseract import pytesseract
def get_captcha_number():
    # Defining paths to tesseract.exe
    # and the image we would be using
    get_captcha_image()
    path_to_tesseract = r"C:\Program Files\Tesseract-OCR\tesseract.exe"
    image_path = r"C:\Users\euh27\Untitled Folder 3\image.jpg"

    # Opening the image & storing it in an image object
    img = Image.open(image_path)

    # Providing the tesseract executable
    # location to pytesseract library
    pytesseract.tesseract_cmd = path_to_tesseract

    # Passing the image object to image_to_string() function
    # This function will extract the text from the image
    text = pytesseract.image_to_string(img)

    # Displaying the extracted text
    return text[:-1]

In [8]:
driver = webdriver.Chrome(ChromeDriverManager().install())



====== WebDriver manager ======
Current google-chrome version is 98.0.4758
Get LATEST chromedriver version for 98.0.4758 google-chrome
Driver [C:\Users\euh27\.wdm\drivers\chromedriver\win32\98.0.4758.80\chromedriver.exe] found in cache


In [9]:
driver.get("https://edugate.ksu.edu.sa/ksu/ui/guest/timetable/index/mainScheduleTreeCoursesIndex.faces")

In [10]:
driver.find_element_by_id('myForm:timetableCrsId').click()

In [11]:
Select(driver.find_element_by_id('myForm:campusGenderCodeItem')).select_by_visible_text('ذكر')

In [12]:
Select(driver.find_element_by_id('myForm:degreeCodeItem')).select_by_visible_text('البكالوريوس')

In [13]:
Select(driver.find_element_by_id('myForm:campusCodeItem')).select_by_visible_text('الرياض- طلاب')

In [14]:
Select(driver.find_element_by_id('myForm:facultyCodeItem')).select_by_visible_text('علوم الحاسب والمعلومات')

In [15]:
dept = 'الكل'
Select(driver.find_element_by_id('myForm:deptCodeItem')).select_by_visible_text(f'{dept}')

In [24]:
captcha = driver.find_element_by_id('myForm:capatchaId')
captcha.send_keys('')
captcha.send_keys(get_captcha_number())
captcha.send_keys(Keys.ENTER)

In [25]:
driver.find_element_by_xpath('//*[@id="myForm"]/table[4]/tbody/tr[1]/td/table/tbody/tr/td/table/tbody/tr[5]/td[6]/a[2]/table/tbody/tr/td[1]/img').click()

In [26]:
content = driver.page_source
soup = BeautifulSoup(content,features='lxml')
rt= soup.find(id="myForm:timetable")
# rt

In [27]:
trs = rt.find_all('tr')
print(len(trs[1:]))

1335


In [28]:
def press_details(index=1):
    details_selector = '#myForm\:timetable > tbody > tr:nth-child({}) > td:nth-child(8) > a'
    details_button = WebDriverWait(driver,10).until(EC.element_to_be_clickable((By.CSS_SELECTOR,details_selector.format(index))))
    details_button.click()
# press_details(3)

In [29]:
def press_details_js(index=0):
    driver.execute_script(f"showToolTipDilaog(event,'{index}');")
# press_details()

In [30]:
def get_times():
#     times = driver.find_element_by_id('sectionDiv').find_elements_by_tag_name('nobr')
    times = WebDriverWait(driver,10).until(EC.element_to_be_clickable((By.ID,'sectionDiv'))).find_elements_by_tag_name('nobr')
    if not times:
        return None
    n=len(times)//3
    return [times[i*3].text for i in range(n)], [times[1+i*3].text for i in range(n)] , [times[2+i*3].text for i in range(n)]
# tmp = {}
# tmp['days'], tmp['time'], tmp['location'] = get_times()
# print(tmp)

In [31]:
def get_final_info():
    return driver.find_element_by_id('examDayDiv').text,driver.find_element_by_id('examTimeDiv').text,driver.find_element_by_id('examDateDiv').text
# print(get_final_info())

In [32]:
def get_instructor():
    return driver.find_element_by_id('instructorDiv').text
# print(get_instructor())

In [33]:
def close():
    driver.find_element_by_xpath('//*[@id="dlg"]/div[1]/a/span').click()

In [34]:
ctrs = trs[1:]
# print(ctrs)
len(ctrs)

1335

In [37]:
res = []
x =0
for i in ctrs:
    tds = i.find_all('td');
    if 'مشروع' in tds[3].text or 'تدريب' in tds[3].text :
        try: 
            close()
        except:
            pass
        x+=1
        continue
    tmp = {
        'cid': tds[0].text,
        'cname': tds[1].text,
        'section': tds[2].text,
        'activity': tds[3].text,
        'chours': tds[4].text,
        'gender': tds[5].text,
        'status': tds[6].text
    }
    press_details_js(x)
    driver.implicitly_wait(5)
    times = get_times()
    
    tmp['days'],tmp['time'],tmp['location']=(times) if times else ('','','')
    
    final_info = get_final_info()
    tmp['fexamday'],tmp['fexamtime'],tmp['fexamdate']=final_info
    
    tmp['instructor'] = get_instructor()
    driver.implicitly_wait(5)
    close()
    driver.implicitly_wait(5)
    res.append(tmp)
#     print(res)
    x+=1

In [38]:
len(res)

1153

In [39]:
import csv
keys = res[0].keys()

with open('all_ccis_courses.csv', 'w',encoding='utf-8-sig', newline='') as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(res)